In [2]:
#files to open: train and test data,

from google.colab import files

uploaded = files.upload()


Saving test_y1_v3.csv to test_y1_v3.csv
Saving test_y1_v8.csv to test_y1_v8.csv
Saving test_y1_v7.csv to test_y1_v7.csv


# Imports and functions

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import class_weight
from sklearn.metrics import recall_score, precision_score, confusion_matrix

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation

Using TensorFlow backend.


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_loss(epochs, loss, val_loss):
    plt.figure(figsize=(10, 7))
    plt.plot(np.arange(epochs), loss, label='loss')
    plt.plot(np.arange(epochs), val_loss, label='val_loss')
    plt.legend()
    plt.grid(True)
def acc_graph(epochs, train_acc, val_acc):
    x_axis = np.linspace(1, epochs, num=epochs)
    
    plt.figure(figsize=(10, 7))
    plt.plot(x_axis, train_acc, color='g', lw=3, alpha=0.7, label='Train Accuracy')
    plt.plot(x_axis, val_acc, color='orange', lw=3, alpha=0.7, label='Val Accuracy')
    plt.title('Accuracy graph')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)

In [0]:
def calculation_classes(y, y_test = None):
    print ('\t\tTRAIN DATA\tTEST DATA')
    print ('Negative\t', np.sum(y==0),'  ',
           round(100*np.sum(y==0)/y.shape[0],2),'%',
           '\t',np.sum(y_test==0),'  ',
           round(100*np.sum(y_test==0)/y_test.shape[0],2),'%'
           '\nPositive\t ', np.sum(y==1),' ',
           round(100*np.sum(y==1)/y.shape[0],2),'%'
           '\t',np.sum(y_test==1),' ',
           round(100*np.sum(y_test==1)/y_test.shape[0],2),'%'
      )

In [0]:
def scale_0_1_for_samples(X):
  mm = MinMaxScaler()

  for i in range(X.shape[0]):
    line = X[i]
    #print(line)
  
    line = line.reshape(-1, 1)
    X[i] = mm.fit_transform(line).reshape(1,-1)
  return X
 
def scale_standard_for_1sample(X):
  ss = StandardScaler()

  for i in range(X.shape[0]):
    line = X[i]
    X[i] = ss.fit_transform(line.reshape(-1,1)).reshape(1,-1)
  return X

def scale_substraction_mean_for_1sample(X):
  for i in range(X.shape[0]):
    line = X[i]
    #print(line)
    mean = np.mean(line)
    #print(mean)
    X[i] = line - mean 
    #if min(X[i]) < 0:
    #  X[i] -= min(X[i])*1.2
    #print(X[i])
    
  return X
    

In [0]:
def get_mean_for_features(X):
  m = X.shape[0]
  X_mean = np.zeros((m,1))
  for i in range(m):
    X_mean[i] = np.mean(X[i,:], axis=0)
  return X_mean

def scale_standard_for_samples(X, X_mean):
  ss = StandardScaler()
  ss.fit(X_mean)

  X = ss.transform(X)
  return X

In [0]:
def make_predictions(X, y, model, X_ini,out=False,):
  preds = model.predict(X)
  y_predict = np.where(preds > 0.5, 1, 0)
  recall = recall_score(y, y_predict)
  print('Recall = ', recall, 'Precision = ', precision_score(y, y_predict))
  cm  = confusion_matrix(y, y_predict)
  print(cm)
  if out is True:
    for i in range(X.shape[0]):
      if abs(y_predict[i] - y[i])!= 0:
        print(y[i], preds[i], X_ini[i], np.ndarray.tolist(X[i]))
    

In [0]:
def calculate_deltas(line):
    deltas = [0]
    for i in range(len(line)-1):
        delta = line[i] - line[i+1]
        deltas.append(delta)
    return deltas

In [0]:
def make_prediction_many_to_many(X,y,model,X_ini, out=False):
  pred = model.predict(X)[:,-1,:]
  #pred = np.where(pred > 0.5, 1, 0)
  pred = np.argmax(pred, axis=-1)
  y_true = np.argmax(y[:,-1,:], axis=-1)
  print(confusion_matrix(y_true, pred))
  if out is True:
    for i in range(X.shape[0]):
      if abs(pred[i] - y_true[i]) != 0:
        full_pred = model.predict(X[i:i+1,:,:])
        print(y[i].tolist(), full_pred.tolist())
        print(X_ini[i], X[i].tolist())
        deltas = calculate_deltas(X_ini[i])
        print('d', deltas)
        print('d2', calculate_deltas(deltas[1:]))
        
        print('---------------------------')
    

In [0]:
def mismatching_pred(X, yoh, X_ini, *classes, model=model, flag_out=False):
  print(classes)
  class1, class0 = classes
  pred = model.predict(X)
  pred = np.argmax(pred, axis=-1)
  y_cl = np.argmax(yoh, axis=-1)
  counter1 = 0
  counter0 = 0
  for i in range(X.shape[0]):
    for j in range(6):
      if y_cl[i,j] == 1 and pred[i,j] == 0:
        counter1 += 1
        if flag_out is True:
          print(counter1)
          print(pred[i].tolist())
          print(y_cl[i].tolist())
          print(X[i,:,0])
          print(X_ini[i])
          d = calculate_deltas(X[i,:,0])
          print(d)
          print(calculate_deltas(d[1:]))
          
          print('--------------------')
          break
      elif y_cl[i,j] == 0 and pred[i,j] == 1:
        counter0 += 1
  print(counter1, counter1/class1)
  if class0 == 0:
    return None
  print(counter0, counter0/class0)
  
def counter_classes(y):
  class1 = 0
  class0 = 0
  
  y_cl = np.argmax(y, axis=-1)
  for line in y_cl:
    for el in line:
      if el == 1:
        class1 += 1
      else:
        class0 += 1
  
  print('1:', class1)
  print('0:', class0)
  
  return class1, class0

In [0]:
from keras.utils import to_categorical

def preprocess_data(Y):
    num = 2
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=num), Y)))

    return Yoh


In [0]:
x_columns = ['Y1','Y2','Y3','Y4','Y5','Y6']
y_columns = ['1','2','3','4','5','6']

def import_train_set(*file_names):
  x_columns = ['Y1','Y2','Y3','Y4','Y5','Y6']
  y_columns = ['1','2','3','4','5','6']
  
  X = np.zeros((0,6))
  y = np.zeros((0,6))
  for file in file_names:
    data_train = pd.read_csv(file)
    X = np.vstack((X, data_train.drop(x_columns, axis='columns').values))
    y = np.vstack((y, data_train.drop(y_columns, axis='columns').values))
  
  X_ini = X.copy()
  
  return X,y, X_ini

# Import data sets

In [0]:
train_files = ['train_y1_v8.csv',
               'train_y0_rand.csv',
               'train_y0_change_1_to_close.csv']

In [0]:
X, y, X_ini = import_train_set(*train_files)

In [0]:
scale_X = scale_substraction_mean_for_1sample

In [17]:
X = scale_X(X)

X_train,X_dev,y_train,y_dev = train_test_split(X, y, test_size=0.2, \
                                               shuffle=True, stratify=y)

'''
class_weights_train = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train[:,-1]),
                                                 y_train[:,-1])
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y[:,-1]), y[:,-1])
print(class_weights)
print(class_weights[0]/(class_weights[1]+class_weights[0]))
'''
calculation_classes(y[:,-1], y_dev[:,-1])

X = np.expand_dims(X, axis=2)
X_train = np.expand_dims(X_train, axis=2)
X_dev = np.expand_dims(X_dev, axis=2)
print(X_train.shape)

		TRAIN DATA	TEST DATA
Negative	 43267    81.79 % 	 8654    81.79 %
Positive	  9636   18.21 %	 1927   18.21 %
(42322, 6, 1)


In [18]:
yoh = preprocess_data(y)
yoh_train = preprocess_data(y_train)
yoh_dev = preprocess_data(y_dev)
print(yoh.shape)

(52903, 6, 2)


In [0]:
data_v4_test = pd.read_csv('test_v4_7615_to_prediction.csv')
X_v4 = data_v4_test.drop(x_columns, axis='columns').values
y_v4 = data_v4_test.drop(y_columns, axis='columns').values
test_v4 =  X_v4.copy()

X_v4 = scale_X(X_v4)
X_v4 = np.expand_dims(X_v4, axis=2)

In [0]:
data_v7 = pd.read_csv('test_y1_v7.csv')

X_v7 = data_v7.drop(x_columns, axis='columns').values
test_v7 =  X_v7.copy()
y_v7 = data_v7.drop(y_columns, axis='columns').values

X_v7 = scale_X(X_v7)
X_v7 = np.expand_dims(X_v7, axis=2)
#y_v7 = np.expand_dims(y_v7, axis=2)
y_v7 = preprocess_data(y_v7)


In [0]:
data_v8 = pd.read_csv('test_y1_v8.csv')

X_v8 = data_v8.drop(x_columns, axis='columns').values
test_v8 =  X_v8.copy()
y_v8 = data_v8.drop(y_columns, axis='columns').values

X_v8 = scale_X(X_v8)
X_v8 = np.expand_dims(X_v8, axis=2)
#y_v8 = np.expand_dims(y_v8, axis=2)
y_v8 = preprocess_data(y_v8)

In [0]:
data_v3 = pd.read_csv('test_y1_v3.csv')

X_v3 = data_v3.drop(x_columns, axis='columns').values
test_v3 =  X_v3.copy()
y_v3 = data_v3.drop(y_columns, axis='columns').values

X_v3 = scale_X(X_v3)
X_v3 = np.expand_dims(X_v3, axis=2)
#y_v3 = np.expand_dims(y_v3, axis=2)
y_v3 = preprocess_data(y_v3)


# NN on train/dev sets

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, GRU, LSTM, TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


def model_GRU(input_shape, Ty, hidden_size):
    i = Input(shape=input_shape, dtype='float32')
    X = GRU(hidden_size, return_sequences=False)(i)
    
    X = Dense(Ty, activation='sigmoid')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model
  
def model_LSTM(hidden_size):  
  model = Sequential()
  model.add(LSTM(hidden_size, return_sequences=False, input_shape=(Tx,1)))
  model.add(Dense(units=1, activation='sigmoid'))
  
  return model

def model_LSTM_many_to_many(hidden_size):
  X = Input(shape=(6, 1))
  h = LSTM(hidden_size, return_sequences=True)(X)
  output = Dense(units=2, activation='softmax')(h)        
    
  model = Model(inputs=X, outputs=output)
  
  return model

def model_LSTM_many_to_many_dence2(hidden_size, dence2):
  X = Input(shape=(6, 1))
  h = LSTM(hidden_size, return_sequences=True)(X)
  print(h.shape)
  h = Dense(units=dence2, activation='tanh')(h) 
  print(h.shape)
  output = Dense(units=2, activation='softmax')(h)        
    
  model = Model(inputs=X, outputs=output)
  
  return model


def model_LSTM_many_to_many_TD(hidden_size):
  X = Input(shape=(6, 1))
  h = LSTM(hidden_size, return_sequences=True)(X)
  output = TimeDistributed(Dense(units=2, activation='softmax'))(h)        
    
  model = Model(inputs=X, outputs=output)
  
  return model

hidden_size = 16
LSTM_cell = LSTM(hidden_size, return_state = True)

def model_LSTM_many_to_many_0(hidden_size):
    X = Input(shape=(6, 1))
    s0 = Input(shape=(hidden_size,), name='s0')
    c0 = Input(shape=(hidden_size,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    for t in range(6):
        s, _, c = LSTM_cell(X, initial_state=[s,c])
        
        out = Dense(units=2, activation='softmax')(s)
        
        outputs.append(out)
    
    model = Model(inputs=[X, s0, c0], outputs=outputs)
   
    return model

In [0]:
# For ready model
#GRU_model = model_GRU((Tx,1),Ty)#GRU_model.summary()
#GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


#GRU_model.load_weights('LSTM_sequence_weights.h5')

In [25]:
hidden_size = 16
epochs = 5
batch_size = 32

print(X_train.shape)

(42322, 6, 1)


In [29]:
model = model_LSTM_many_to_many(hidden_size)
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999,decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

logs = model.fit(X_train, yoh_train, epochs=epochs, batch_size=batch_size, shuffle=True,\
                     validation_data=(X_dev, yoh_dev), verbose=2) #, class_weight=class_weights_train, \
                   # initial_epoch=0) 

Train on 42322 samples, validate on 10581 samples
Epoch 1/5
 - 7s - loss: 0.3190 - acc: 0.8463 - val_loss: 0.2847 - val_acc: 0.8586
Epoch 2/5
 - 5s - loss: 0.2799 - acc: 0.8597 - val_loss: 0.2789 - val_acc: 0.8589
Epoch 3/5
 - 5s - loss: 0.2752 - acc: 0.8607 - val_loss: 0.2761 - val_acc: 0.8596
Epoch 4/5
 - 5s - loss: 0.2721 - acc: 0.8619 - val_loss: 0.2743 - val_acc: 0.8600
Epoch 5/5
 - 5s - loss: 0.2702 - acc: 0.8621 - val_loss: 0.2720 - val_acc: 0.8575


In [30]:
model_TD = model_LSTM_many_to_many_TD(hidden_size)
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999,decay=0.0)
model_TD.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

logs = model_TD.fit(X_train, yoh_train, epochs=epochs, batch_size=batch_size, shuffle=True,\
                     validation_data=(X_dev, yoh_dev), verbose=2) #, class_weight=class_weights_train, \
                   # initial_epoch=0) 

Train on 42322 samples, validate on 10581 samples
Epoch 1/5
 - 6s - loss: 0.3166 - acc: 0.8465 - val_loss: 0.2886 - val_acc: 0.8569
Epoch 2/5
 - 5s - loss: 0.2810 - acc: 0.8588 - val_loss: 0.2774 - val_acc: 0.8605
Epoch 3/5
 - 5s - loss: 0.2767 - acc: 0.8603 - val_loss: 0.2833 - val_acc: 0.8596
Epoch 4/5
 - 5s - loss: 0.2728 - acc: 0.8611 - val_loss: 0.2804 - val_acc: 0.8524
Epoch 5/5
 - 5s - loss: 0.2704 - acc: 0.8619 - val_loss: 0.2716 - val_acc: 0.8610


In [97]:
model_dence2 = model_LSTM_many_to_many_dence2(hidden_size=32, dence2=16)
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999,decay=0.0)
model_dence2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

logs = model_dence2.fit(X_train, yoh_train, epochs=epochs, batch_size=batch_size, shuffle=True,\
                     validation_data=(X_dev, yoh_dev), verbose=2) #, class_weight=class_weights_train, \
                   # initial_epoch=0) 

(?, ?, 32)
(?, 6, 16)
Train on 42322 samples, validate on 10581 samples
Epoch 1/5
 - 9s - loss: 0.3070 - acc: 0.8478 - val_loss: 0.2795 - val_acc: 0.8590
Epoch 2/5
 - 6s - loss: 0.2770 - acc: 0.8595 - val_loss: 0.2740 - val_acc: 0.8617
Epoch 3/5
 - 6s - loss: 0.2731 - acc: 0.8606 - val_loss: 0.2703 - val_acc: 0.8616
Epoch 4/5
 - 6s - loss: 0.2700 - acc: 0.8611 - val_loss: 0.2714 - val_acc: 0.8615
Epoch 5/5
 - 6s - loss: 0.2665 - acc: 0.8622 - val_loss: 0.2627 - val_acc: 0.8639


In [131]:
l_rate = 0.0002
decay = 0.015
rate = l_rate
for i in range(n_epochs):
  i += 1
  rate = rate/(1+decay * i)
print(rate)
opt = Adam(lr=l_rate, beta_1=0.9, beta_2=0.999, decay=decay)
GRU_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

9.124217057522553e-05


In [132]:
n_epochs = 10
logs = model_dence2.fit(X_train, yoh_train, epochs=n_epochs, batch_size=batch_size, shuffle=True,\
                        validation_data=(X_dev, yoh_dev), verbose=2) #, class_weight=class_weights_train, \

Train on 42322 samples, validate on 10581 samples
Epoch 1/10
 - 8s - loss: 0.0117 - acc: 0.9960 - val_loss: 0.0133 - val_acc: 0.9957
Epoch 2/10
 - 6s - loss: 0.0104 - acc: 0.9964 - val_loss: 0.0174 - val_acc: 0.9936
Epoch 3/10
 - 6s - loss: 0.0107 - acc: 0.9962 - val_loss: 0.0160 - val_acc: 0.9941
Epoch 4/10
 - 6s - loss: 0.0099 - acc: 0.9965 - val_loss: 0.0085 - val_acc: 0.9973
Epoch 5/10
 - 6s - loss: 0.0114 - acc: 0.9961 - val_loss: 0.0148 - val_acc: 0.9949
Epoch 6/10
 - 6s - loss: 0.0099 - acc: 0.9965 - val_loss: 0.0113 - val_acc: 0.9963
Epoch 7/10
 - 6s - loss: 0.0102 - acc: 0.9964 - val_loss: 0.0120 - val_acc: 0.9956
Epoch 8/10
 - 6s - loss: 0.0110 - acc: 0.9960 - val_loss: 0.0180 - val_acc: 0.9937
Epoch 9/10
 - 6s - loss: 0.0098 - acc: 0.9966 - val_loss: 0.0303 - val_acc: 0.9916
Epoch 10/10
 - 6s - loss: 0.0101 - acc: 0.9964 - val_loss: 0.0126 - val_acc: 0.9953


In [67]:
print(X.shape)
#print(pred.shape, y_cl.shape)

(52903, 6, 1)


In [71]:
class1, class0 = counter_classes(yoh)
class1_v3, class0_v3 = counter_classes(y_v3)
class1_v7, class0_v7 = counter_classes(y_v7)
class1_v8, class0_v8 = counter_classes(y_v8)

1: 160257
0: 157161
1: 618
0: 0
1: 1044
0: 0
1: 606
0: 0


In [0]:
out = False

In [126]:
mismatching_pred(X, yoh, X_ini, *[class1, class0], model=model_dence2, flag_out=False)

(160257, 157161)
761 0.004748622525069108
475 0.0030223783254115207


In [127]:
make_prediction_many_to_many(X,yoh,out=out,model=model_dence2,X_ini=X_ini)

[[43267     0]
 [    1  9635]]


In [128]:
make_prediction_many_to_many(X_v3,y_v3,out=out,model=model_dence2,X_ini=test_v3)
mismatching_pred(X_v3, y_v3, test_v3, *[class1_v3, class0_v3],model=model_dence2, flag_out=out)


[[ 0  0]
 [27 76]]
(618, 0)
83 0.1343042071197411


In [129]:
make_prediction_many_to_many(X_v8, y_v8, out=out, model=model_dence2, X_ini=test_v8)
mismatching_pred(X_v8, y_v8, test_v8, *[class1_v8, class0_v8], model=model_dence2, flag_out=out)

[[ 0  0]
 [ 6 95]]
(606, 0)
25 0.041254125412541254


In [130]:
make_prediction_many_to_many(X_v7, y_v7, out=out, model=model_dence2, X_ini=test_v7)
mismatching_pred(X_v7, y_v7, test_v7, *[class1_v7, class0_v7], model=model_dence2, flag_out=out)

[[  0   0]
 [ 29 145]]
(1044, 0)
98 0.09386973180076628


In [0]:
 make_prediction_many_to_many(X_v4, y_v4, out=out, model=GRU_model, X_ini=test_v4)

In [0]:
make_predictions(X,y, GRU_model,X_ini, out=False)
make_predictions(X_v7v8, y_v7v8, out=True, model=GRU_model, X_ini=test_v7v8)
make_predictions(X_v3, y_v3, out=True, model=GRU_model, X_ini=test_v3)
make_predictions(X_v4, y_v4, out=True, model=GRU_model, X_ini=test_v4)

In [0]:
#plot_loss(epochs, logplot_loss(epos.history.get('loss'), logs.history.get('val_loss'))

In [0]:
#acc_graph(epochs, logs.history.get('acc'), logs.history.get('val_acc'))

In [0]:
GRU_model.save_weights('LSTM_sequence_weights.h5')

In [0]:
files.download('LSTM_sequence_weights.h5')

# Final fit NN on all train set

In [0]:
Tx=  6
Ty = 1
epochs = 250
batch_size = 32

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
LSTM_model.fit(X, y , epochs=epochs, batch_size=batch_size, shuffle=True,\
               verbose=0, class_weight=class_weights)
LSTM_model.save_weights('sources/LSTM_sequence_weights.h5')

In [0]:
print(LSTM_model.summary())
test_preds = LSTM_model.predict(X_test)

In [0]:
for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)

In [0]:
LSTM_model = model((Tx,1),Ty)
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('LSTM_sequence_weights.h5')

In [0]:
test_preds = LSTM_model.predict(X_test)

for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)